In [1]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pierre!!"

In [2]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

2.5.5


# Parse data and create a NetworkX graph

In [3]:
import networkx as nx

In [4]:
input_file = 'input.txt'
lines = open(input_file, 'r').readlines()
lines = [line. strip() for line in lines]

In [5]:
def make_id(slot):
    return 1000 * slot[0] + slot[1]

In [6]:
nxG = nx.DiGraph()

for j, line in enumerate(lines):
    for i, sym in enumerate(line):
        curr = (i, j)
        label = {'S':'Source', '.':'Free', '#':'Rock'}[sym]
        if label == 'Source':
            source = curr
            print('label',curr)
        if i < len(line)-1:
            east_neighbor = (i+1, j)
        else:
            east_neighbor = None
        if j < len(lines)-1:
            south_neighbor = (i, j+1)
        else:
            south_neighbor = None

        nxG.add_node(make_id(curr), labels=[label], i=curr[0], j=curr[1])

        if(east_neighbor):
            nxG.add_edge(make_id(curr), make_id(east_neighbor), relationshipType='EAST')
            nxG.add_edge(make_id(east_neighbor), make_id(curr), relationshipType='WEST')
        if(south_neighbor):
            nxG.add_edge(make_id(curr), make_id(south_neighbor), relationshipType='SOUTH')
            nxG.add_edge(make_id(south_neighbor),make_id(curr), relationshipType='NORTH')

label (65, 65)


# Load nx graph to in-memory GDS graph

In [14]:
#G = gds.graph.get("slots")
#gds.graph.drop(G)
G = gds.graph.networkx.load(nxG, "slots")

# Remove obstacles by filtering the graph

In [16]:
#H = gds.graph.get("no-rock-slots")
#gds.graph.drop(H)
gds.graph.filter('no-rock-slots', G,  'n:Source OR n:Free', 'r:EAST OR r:SOUTH OR r:NORTH OR r:WEST')

GraphCreateResult(graph=Graph({'graphName': 'no-rock-slots', 'nodeCount': 15348, 'relationshipCount': 54502, 'database': 'neo4j', 'configuration': {'readConcurrency': 4, 'undirectedRelationshipTypes': [], 'jobId': '9476551a-a585-4050-94c8-7f52ab66aec4', 'logProgress': True, 'query': 'UNWIND $data AS data WITH data, CASE WHEN data[6] THEN data[5] ELSE null END AS sourceNodeLabels, CASE WHEN data[3] THEN data[2] ELSE null END AS relationshipType, CASE WHEN data[10] THEN data[9] ELSE null END AS targetNodeId, \nCASE WHEN data[8] THEN data[7] ELSE null END AS sourceNodeProperties, \nCASE WHEN data[1] THEN data[0] ELSE null END AS relationshipProperties RETURN gds.graph.project($graph_name, data[4], targetNodeId, {sourceNodeLabels: sourceNodeLabels, targetNodeLabels: NULL, sourceNodeProperties: sourceNodeProperties, targetNodeProperties: NULL, relationshipType: relationshipType}, $configuration)', 'inverseIndexedRelationshipTypes': [], 'concurrency': 4, 'creationTime': neo4j.time.DateTime(2

In [17]:
H = gds.graph.get('no-rock-slots')
H.node_count(), H.relationship_count()

(15348, 54502)

We could run gds algorithm on such a graph although not using the DB.
That not convenient here.

# Exporting to a Neo4j DB
An ephemeral db would do if we only wanted read-only (or similar) queries. In this case I will persist to a new DB.


In [19]:
#gds.run_cypher('DROP DATABASE `h-db`')
gds.graph.export(H, dbName="h-db")

dbName                                h-db
graphName                    no-rock-slots
nodeCount                            15348
relationshipCount                    54502
relationshipTypeCount                    4
nodePropertyCount                    61392
relationshipPropertyCount                0
writeMillis                           9764
Name: 0, dtype: object

In [20]:
gds.run_cypher('CREATE DATABASE `h-db`')

""


# Using the Neo4j DB with cypher 

In [21]:
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), database="h-db")

In [22]:
gds.run_cypher('MATCH (n) SET n.available_at_step = [], n:Point')

""


In [23]:
gds.run_cypher('MATCH (n:Source) SET n.available_at_step = [0]')

""


In [24]:
gds.run_cypher('CREATE INDEX point_available_at_step IF NOT EXISTS FOR (n:Point) ON (n.available_at_step)')

""


In [25]:
gds.run_cypher("""
UNWIND range(0,64-1) AS i
CALL {
    WITH i
    MATCH (p:Point WHERE i IN p.available_at_step)-->(neigh)
    SET neigh.available_at_step = apoc.coll.union(neigh.available_at_step, [i+1] )
} IN TRANSACTIONS OF 1 ROWS
""")

""


In [26]:
gds.run_cypher("""
MATCH (n:Point)
WHERE 64 in n.available_at_step
RETURN count(n) AS part1
""")

,part1
0,3768
